In [14]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [15]:
import cv2

In [16]:
img = cv2.imread(r"C:\Users\supichik\Downloads\face-mask-dataset\Dataset\with_mask\4-with-mask.jpg")#reads an image as nump array

In [17]:
print(img.shape)#gives image shape (rows,columns,channels)

(429, 311, 3)


Creating a dataset with each image as numpy array and its corresponding label

In [18]:
import os

targets = ["with_mask","without_mask"]

path = r"C:\Users\supichik\Downloads\face-mask-dataset\Dataset"

a=0

data = []

for target in targets:
    fold_path = os.path.join(path,target)#joins the path + each element in targets
    
    for image in os.listdir(fold_path):
        img_path = os.path.join(fold_path,image)#joins the updated path+image url
        img_array = cv2.imread(img_path)#reading an image as numpy array
        resized_img_array = cv2.resize(img_array,(224,224))#resizing the image to (224,224,3) to give as input to our model
        label = targets.index(target)
        data.append([resized_img_array,label])#creating a list contians image array and its corresponding label and appending to data variable
        


        

In [19]:
#Data list has an arrays and its labels in sequential format hence we are shuffling it
import random
random.shuffle(data)


# l = [[1,'a'],[2,'b']]

# fe1 = []
# la1 = []
# for fe,la in l:
#     fe1.append(fe)
#     la1.append(la)
# print(fe1)
# print(la1)

# o/p :
# [1, 2]
# ['a', 'b']

#creating the X & Y which contains image arrays & corresponding labels

X = []
Y = []

for feature,label in data:
    X.append(feature)
    Y.append(label)


In [20]:
#converting both X and Y to numpy arrays
import numpy as np
X = np.array(X)
Y = np.array(Y)

In [21]:
X.shape#(no of images,rows,columns,channels)

(1508, 224, 224, 3)

In [22]:
Y

array([0, 0, 0, ..., 0, 0, 0])

In [23]:
len(X)

1508

In [24]:
len(Y)

1508

In [25]:
#Standardizing the X so that all the elements in the array will scale down between 0 and 1

X = X/255

In [26]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [27]:
#splitting the X,Y into train and test data with stratify on Y
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y)

In [28]:
X_train.shape

(1206, 224, 224, 3)

In [29]:
Y_train.shape

(1206,)

In [30]:
X_test.shape

(302, 224, 224, 3)

In [31]:
Y_test.shape

(302,)

In [32]:
pip install pandas


In [33]:
#checking wether stratification is performed or not after train_test_split
import pandas as pd
r = Y_train
pd.Series(r).value_counts()

0    604
1    602
dtype: int64

In [34]:
m = Y_test
pd.Series(m).value_counts()

0    151
1    151
dtype: int64

# BUILDING A MODEL

In [35]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [36]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [37]:
pip install tensorflow

# VGG 16

In [80]:
import tensorflow as tf
from keras.applications.vgg16 import VGG16

In [81]:
vgg=VGG16()#creating an object of VGG16(this downlowds the pre-trained VGG16 on ImageNet Datset)

In [82]:
vgg.summary()#tells the summary of the pre-trained VGG16 model on ImageNet

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [83]:
from keras import Sequential

In [84]:
model = Sequential()#creating our own model

In [85]:
for layer in vgg.layers[:-1]:#excluding the last layer in Pre-trained VGG16
    model.add(layer)

In [86]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [87]:
for layer in model.layers:#making the trianable parameters to freeze so that we can use the default parameters which got trained from pre-trained VGG16
    layer.trainable=False

In [88]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [89]:
from keras.layers import Dense

In [90]:
model.add(Dense(1,activation='sigmoid'))#adding sigmoid as last layer


In [91]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [92]:
model.compile(optimizer="Adam",loss = "binary_crossentropy",metrics=['accuracy'])#compiling the model

In [93]:
model.fit(X_train,Y_train,epochs=5,validation_data=(X_test,Y_test))#fitting the model to our train data set

Epoch 1/5
38/38 [==============================] - 514s 13s/step - loss: 0.6528 - accuracy: 0.5994 - val_loss: 0.3916 - val_accuracy: 0.8907
Epoch 2/5
38/38 [==============================] - 435s 11s/step - loss: 0.3577 - accuracy: 0.9062 - val_loss: 0.2900 - val_accuracy: 0.9172
Epoch 3/5
38/38 [==============================] - 458s 12s/step - loss: 0.2558 - accuracy: 0.9403 - val_loss: 0.2360 - val_accuracy: 0.9437
Epoch 4/5
38/38 [==============================] - 442s 12s/step - loss: 0.2034 - accuracy: 0.9496 - val_loss: 0.2049 - val_accuracy: 0.9570
Epoch 5/5
38/38 [==============================] - 491s 13s/step - loss: 0.1746 - accuracy: 0.9557 - val_loss: 0.1809 - val_accuracy: 0.9536


In [94]:

def prediction(imge,models):
    model=models
    y_pred = model.predict_classes(imge.reshape(1,224,224,3))#predicts the lable for the given frame
    return y_pred[0][0]

In [95]:
def draw_txt(image,text,org,font,fontscale,color,thickness):
    image = cv2.putText(image, text, org, font, fontscale, color, thickness)#draws text on image
    return image

In [96]:
def capturing_on_video(models):
    cap = cv2.VideoCapture(0)#helps to use the built-in camera, if we want to use connected camers we can pass arguments linke 1,2,3
    #1 -says 1st connected camera to the device, 2-says 2nd connected camera to our device...so on.
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()#reads the image in numpy array captured by the video from our built-in-camera
        resized_frame = cv2.resize(frame,(224,224))#resizing the captured image
        y_pred = prediction(resized_frame,models)#predicting its label/class
        if y_pred==0:
            image = draw_txt(frame,"MASK",(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)#if predicted label is 0 it adds text(mask) on the captured image
        else:
            image = draw_txt(frame,"NO_MASK",(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)#if predicted label is 1 it adds text(no_mask) on the captured image
        cv2.imshow("video",image)#display the image on window
        if cv2.waitKey(1) & 0xFF == ord('q'):#cv2.waitKey(1) -  displays for 1 millisecond,0xFF == ord('q') - q key from key-board 
            break
    cap.release()#releases the captured video
    cv2.destroyAllWindows()#destroys all the windows

In [97]:
pip install h5py


In [98]:
from keras.models import model_from_json

In [99]:
# serialize model to JSON/converting model to json object
model_json = model.to_json()
with open("vgg_last_layer.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [100]:
#loading model from json file
with open("vgg_last_layer.json", "r") as json_file:
    loaded_model_json = json_file.read()
vgg_last_lay_loaded_model = model_from_json(loaded_model_json)
# load weights into new model
vgg_last_lay_loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [101]:
# #this helps us to print 0/1 as output based on the mask & no_mask on video
# def capturing_on_video(models):
#     cap = cv2.VideoCapture(0)
#     while True:
#         ret, frame = cap.read()
#         resized_frame = cv2.resize(frame,(224,224))
#         print(prediction(resized_frame,models))
#         cv2.imshow("video",frame)
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
#     cap.release()
#     cv2.destroyAllWindows()



In [102]:
capturing_on_video(vgg_last_lay_loaded_model)

Testing the model with sample image

In [49]:
img = cv2.imread(r"C:\Users\supichik\Downloads\face-mask-dataset\Dataset\with_mask\1-with-mask.jpg")

In [50]:
print(img)

[[[248 250 250]
  [248 250 250]
  [248 250 250]
  ...
  [249 251 251]
  [249 251 251]
  [249 251 251]]

 [[248 250 250]
  [248 250 250]
  [248 250 250]
  ...
  [249 251 251]
  [249 251 251]
  [249 251 251]]

 [[248 250 250]
  [248 250 250]
  [248 250 250]
  ...
  [249 251 251]
  [249 251 251]
  [249 251 251]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [ 20  22  22]
  [ 19  21  21]
  [ 18  20  20]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [ 20  22  22]
  [ 19  21  21]
  [ 18  20  20]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [ 20  22  22]
  [ 19  21  21]
  [ 18  20  20]]]


In [51]:
img = img/255


In [52]:
resimg = cv2.resize(img,(224,224))

In [53]:
resimg.shape

(224, 224, 3)

In [54]:
print(prediction(resimg,vgg_last_lay_loaded_model))

0


# VGG16 last 3 layers

In [55]:
#Building another VGG16 model with last 3 dense layers trianable
model_1 = Sequential()

In [56]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [57]:
for layer in vgg.layers[:-3]:#excluding the last 3 layers of pre-trained vgg16
    model_1.add(layer)


In [58]:
model_1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [59]:
for layer in model_1.layers:#frozening the all the trainable parameters
    layer.trainable=False


In [60]:
model_1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [61]:
#adding 3 dense layers in substitue of pre-trained model
model_1.add(Dense(512, activation='relu'))
model_1.add(Dense(512,activation='relu'))
model_1.add(Dense(1,activation='sigmoid'))
model_1.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [62]:
model_1.compile(optimizer="Adam",loss = "binary_crossentropy",metrics=['accuracy'])

In [63]:
model_1.fit(X_train,Y_train,epochs=10,validation_data=(X_test,Y_test))

Epoch 1/10
38/38 [==============================] - 600s 15s/step - loss: 0.7707 - accuracy: 0.8419 - val_loss: 0.0904 - val_accuracy: 0.9735
Epoch 2/10
38/38 [==============================] - 441s 12s/step - loss: 0.0154 - accuracy: 0.9958 - val_loss: 0.0952 - val_accuracy: 0.9735
Epoch 3/10
38/38 [==============================] - 459s 12s/step - loss: 0.0883 - accuracy: 0.9783 - val_loss: 0.1762 - val_accuracy: 0.9536
Epoch 4/10
38/38 [==============================] - 460s 12s/step - loss: 0.0057 - accuracy: 0.9958 - val_loss: 0.0405 - val_accuracy: 0.9934
Epoch 5/10
38/38 [==============================] - 451s 12s/step - loss: 1.2870e-04 - accuracy: 1.0000 - val_loss: 0.0463 - val_accuracy: 0.9934
Epoch 6/10
38/38 [==============================] - 430s 11s/step - loss: 5.7738e-05 - accuracy: 1.0000 - val_loss: 0.0471 - val_accuracy: 0.9934
Epoch 7/10
38/38 [==============================] - 424s 11s/step - loss: 3.6677e-05 - accuracy: 1.0000 - val_loss: 0.0481 - val_accuracy: 0

In [64]:
# serialize model to JSON/converting model to json object
model1_json = model_1.to_json()
with open("vgg_last_3_layer.json", "w") as json_file:
    json_file.write(model1_json)
# serialize weights to HDF5
model_1.save_weights("model1.h5")
print("Saved model to disk")

Saved model to disk


In [65]:
#loading model from json file
with open("vgg_last_3_layer.json", "r") as json_file:
    loaded_model1_json = json_file.read()
vgg_last_3_lay_loaded_model = model_from_json(loaded_model1_json)
# load weights into new model
vgg_last_3_lay_loaded_model.load_weights("model1.h5")
print("Loaded model from disk")

Loaded model from disk


In [66]:
capturing_on_video(vgg_last_3_lay_loaded_model)